52100674 - TRẦN THỊ VẸN

1) Xây dựng ví dụ các mô hình Neural Networks sử dụng Pytorch, trong đó có xử lý phần Overfitting và sử dụng Optimizers khác nhau.

2) Hãy trình bày về Pytorch:

- Những đặc điểm chính khi so sánh với tensorflow

- Những đặc điểm chính trong lập trình với pytorch

- Những class quan trọng trong việc xây dựng mô hình Neural Network

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [49]:
# Load and preprocess MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [50]:
# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [51]:
# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer_Adam = optim.Adam(model.parameters(), lr=0.001)

In [52]:
# Training loop
def train(model, optimizer, criterion, trainloader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:    # Print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0


In [53]:
# Training with different optimizers
print("Training with SGD optimizer:")
train(model, optimizer_SGD, criterion, trainloader)

print("\nTraining with Adam optimizer:")
train(model, optimizer_Adam, criterion, trainloader)

Training with SGD optimizer:
[1,   100] loss: 1.004
[1,   200] loss: 0.506
[1,   300] loss: 0.433
[1,   400] loss: 0.385
[1,   500] loss: 0.349
[1,   600] loss: 0.320
[1,   700] loss: 0.322
[1,   800] loss: 0.302
[1,   900] loss: 0.293
[2,   100] loss: 0.280
[2,   200] loss: 0.252
[2,   300] loss: 0.238
[2,   400] loss: 0.233
[2,   500] loss: 0.217
[2,   600] loss: 0.217
[2,   700] loss: 0.224
[2,   800] loss: 0.215
[2,   900] loss: 0.220
[3,   100] loss: 0.203
[3,   200] loss: 0.198
[3,   300] loss: 0.184
[3,   400] loss: 0.183
[3,   500] loss: 0.186
[3,   600] loss: 0.172
[3,   700] loss: 0.170
[3,   800] loss: 0.167
[3,   900] loss: 0.175
[4,   100] loss: 0.165
[4,   200] loss: 0.156
[4,   300] loss: 0.166
[4,   400] loss: 0.164
[4,   500] loss: 0.156
[4,   600] loss: 0.158
[4,   700] loss: 0.164
[4,   800] loss: 0.157
[4,   900] loss: 0.145
[5,   100] loss: 0.136
[5,   200] loss: 0.156
[5,   300] loss: 0.156
[5,   400] loss: 0.139
[5,   500] loss: 0.135
[5,   600] loss: 0.137
[5,  

**2) Hãy trình bày về Pytorch:**

**Những đặc điểm chính khi so sánh với tensorflow**

**TensorFlow**

Các mô hình TensorFlow thường được tạo ra với sự trợ giúp của Keras.

Lưu ý về Keras, Đây là một thư viện mã nguồn mở chỉ cung cấp các API cấp cao, không giống như TensorFlow cung cấp cả API cấp cao và cấp thấp. Nhưng vì Keras được xây dựng bằng Python, nên việc phát triển các mô hình trong Keras thân thiện với người dùng hơn so với TensorFlow.

Bất kỳ phương pháp nào trong ba cách tiếp cận đều có thể được áp dụng để phát triển các mô hình trong Keras như: Phân lớp, API chức năng và API mô hình tuần tự.

+ Phân lớp: lớp tf.keras.Model có thể được sử dụng để phát triển các mô hình có thể tùy chỉnh về tính hoàn chỉnh của nó và logic chuyển tiếp được triển khai trong phương thức gọi trong khi các lớp được xác định trong phương thức _init_ (). Quan trọng nhất, cách tiếp cận hướng đối tượng giúp tái sử dụng các lớp nhiều lần và xác định chuyển tiếp cực kỳ phức tạp.

+ API chức năng: đây là một cách tiếp cận rất thân thiện với người dùng để phát triển mô hình Mạng Nơ-ron so với Phân lớp theo khuyến nghị của cộng đồng các nhà phát triển. Cách tiếp cận này đòi hỏi một chút ít mã hóa vì đầu vào của lớp trước ngay lập tức được truyền vào và khi layer được xác định. Mô hình được khởi tạo thông qua các tensor đầu vào và đầu ra.

+ API mô hình tuần tự: nó là một dạng rút gọn cho một mô hình có thể huấn luyện, chỉ bao gồm một vài lớp phổ biến và do đó là một cách rất nhỏ gọn và đơn gian để xác định mô hình. Mặt khác, cách tiếp cận này hoạt động rất tốt về mặt hiệu suất khi tạo ra Mạng Nơ-ron đơn giản nhưng việc triển khai Mạng Nơ-ron phức tạp trở nên rất khó khăn trong những từ khác không thể linh hoạt.


**PyTorch**

So với TensorFlow hoặc Keras, chỉ có hai cách tiếp cận để phát triển các mô hình Mạng Nơ-ron với sự trợ giúp của PyTorch: Phân lớp và tuần tự.

+ Phân lớp: việc thực hiện với phương pháp này rất giống với trong TensorFlow (Keras). Ở đây, việc phân lớp được thực hiện thông qua mô đun nn.Model và các layers được xác định trong phương thức _init_ () nhưng việc tạo chuyển tiếp được thực hiện trong phương thức chuyển tiếp thay vì gọi như trong TensorFlow (Keras).

Trong PyTorch cần phải có kích thước hạt nhân chính xác để làm cho nó có mức trung bình toàn cầu, lý do là chỉ có một lớp tổng hợp trung bình có sẵn.

+ Tuần tự: cách tiếp cận này cũng rất giống với cách nó được thực hiện trong TensorFlow (Keras) và được thực hiện thông qua mô-đun Sequential

**Những đặc điểm chính trong lập trình với pytorch**




+ Dynamic Computation Graphs: Một trong những đặc điểm chính của PyTorch là sử dụng dynamic computation graphs. Điều này có nghĩa là đồ thị tính toán có thể thay đổi trong quá trình chạy. Điều này mang lại tính linh hoạt cao cho việc xây dựng và điều chỉnh mô hình, đặc biệt là trong quá trình thử nghiệm và debug.

+ Tích hợp tốt với Python: PyTorch được thiết kế để làm việc tốt với Python và có cú pháp gần gũi với ngôn ngữ này. Điều này làm cho việc xây dựng và debug mô hình trở nên dễ dàng hơn, đặc biệt là đối với những người đã quen thuộc với Python.

+ Hỗ trợ GPU mạnh mẽ: PyTorch cung cấp hỗ trợ mạnh mẽ cho việc tính toán trên GPU, giúp tăng tốc quá trình huấn luyện mô hình. Việc chuyển đổi giữa tính toán trên CPU và GPU cũng được thực hiện một cách dễ dàng thông qua PyTorch.

+ Các utility functions và API đầy đủ: PyTorch cung cấp một loạt các utility functions và API cho việc xử lý dữ liệu, xây dựng mô hình, và tối ưu hóa. Điều này bao gồm các lớp mạng nơ-ron, hàm loss function, optimizer và các công cụ hỗ trợ việc tiền xử lý dữ liệu.

+ Sự phát triển mạnh mẽ của cộng đồng: PyTorch là một dự án mã nguồn mở với một cộng đồng lớn và tích cực phát triển. Điều này có nghĩa là có rất nhiều tài liệu học tập, mã nguồn mẫu và các thư viện bổ sung được phát triển bởi cộng đồng người dùng PyTorch.

**Những class quan trọng trong việc xây dựng mô hình Neural Network**

**Trong việc xây dựng mô hình Neural Network bằng PyTorch:**

torch.nn.Module: Là lớp cơ sở cho tất cả các mô hình trong PyTorch. Các mô hình được xây dựng bằng cách kế thừa từ lớp này và định nghĩa phương thức forward. Trong phương thức forward, người lập trình định nghĩa cách tính toán của mô hình.

torch.nn.Linear: Là lớp thực hiện phép biến đổi tuyến tính, thường được sử dụng cho các layer fully connected trong mạng nơ-ron. Lớp này chấp nhận đầu vào là một tensor có số chiều tùy ý và trả về một tensor với số chiều tương ứng.

torch.nn.Conv2d và torch.nn.ConvTranspose2d: Đây là lớp convolution và deconvolution 2D. Convolution layer thực hiện phép tích chập trên dữ liệu đầu vào, trong khi deconvolution layer thực hiện phép chập ngược lại. Cả hai lớp này thường được sử dụng trong các mạng nơ-ron sâu cho việc xử lý hình ảnh.

torch.nn.ReLU, torch.nn.Sigmoid, torch.nn.Tanh, etc.: Đây là các lớp activation function thường được sử dụng trong các mạng nơ-ron. Chúng được áp dụng sau mỗi layer để tạo ra đầu ra phi tuyến tính và giúp mô hình học được các mối quan hệ phức tạp giữa các đặc trưng đầu vào và đầu ra.

torch.nn.BatchNorm2d và torch.nn.BatchNorm1d: Là lớp Batch Normalization dùng để ổn định và tăng tốc quá trình huấn luyện của mạng nơ-ron. Chúng giúp chuẩn hóa đầu ra của các layer trước khi được đưa vào các activation function, giảm hiện tượng vanishing gradient và tăng tốc độ hội tụ của mô hình.

torch.nn.Dropout: Là lớp dropout, được sử dụng để ngẫu nhiên "tắt" một số đơn vị nơ-ron trong quá trình huấn luyện. Điều này giúp giảm overfitting và làm cho mô hình trở nên tổng quát hơn.

torch.nn.CrossEntropyLoss, torch.nn.MSELoss, etc.: Là các hàm loss function thường được sử dụng để đo lường sự khác biệt giữa dự đoán của mô hình và giá trị thực tế của đầu ra. Các hàm này thường được sử dụng trong quá trình huấn luyện để tính toán loss và điều chỉnh các tham số của mô hình.

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Define model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(0.5)  # Dropout layer

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Training loop
def train(model, optimizer, criterion, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

# Train model
train(model, optimizer, criterion, train_loader)

[1,   100] loss: 1.669
[1,   200] loss: 0.941
[1,   300] loss: 0.778
[1,   400] loss: 0.695
[1,   500] loss: 0.662
[1,   600] loss: 0.622
[1,   700] loss: 0.590
[1,   800] loss: 0.566
[1,   900] loss: 0.555
[2,   100] loss: 0.508
[2,   200] loss: 0.524
[2,   300] loss: 0.520
[2,   400] loss: 0.506
[2,   500] loss: 0.504
[2,   600] loss: 0.459
[2,   700] loss: 0.449
[2,   800] loss: 0.470
[2,   900] loss: 0.463
[3,   100] loss: 0.450
[3,   200] loss: 0.461
[3,   300] loss: 0.454
[3,   400] loss: 0.422
[3,   500] loss: 0.425
[3,   600] loss: 0.436
[3,   700] loss: 0.407
[3,   800] loss: 0.430
[3,   900] loss: 0.453
[4,   100] loss: 0.434
[4,   200] loss: 0.408
[4,   300] loss: 0.437
[4,   400] loss: 0.407
[4,   500] loss: 0.402
[4,   600] loss: 0.404
[4,   700] loss: 0.400
[4,   800] loss: 0.411
[4,   900] loss: 0.380
[5,   100] loss: 0.396
[5,   200] loss: 0.394
[5,   300] loss: 0.390
[5,   400] loss: 0.398
[5,   500] loss: 0.376
[5,   600] loss: 0.397
[5,   700] loss: 0.399
[5,   800] 

**Trong việc xây dựng mô hình Neural Network bằng TensorFlow:**

tf.keras.layers.Dense: Lớp này thực hiện phép biến đổi tuyến tính, thường được sử dụng cho các layer fully connected trong mạng nơ-ron. Đầu vào và đầu ra của nó là các tensor có số chiều tùy ý.

tf.keras.layers.Conv2D và tf.keras.layers.Conv2DTranspose: Đây là lớp convolution và deconvolution 2D. Lớp convolution thực hiện phép tích chập trên dữ liệu đầu vào, trong khi lớp deconvolution thực hiện phép chập ngược lại. Cả hai lớp này thường được sử dụng trong các mạng nơ-ron sâu cho việc xử lý hình ảnh.

tf.keras.layers.Activation: Lớp này áp dụng một hàm activation function lên đầu ra của các layer trước đó. Các activation function thông thường bao gồm ReLU, Sigmoid, Tanh, ...

tf.keras.layers.Dropout: Lớp này thực hiện phép dropout, ngẫu nhiên "tắt" một số đơn vị nơ-ron trong quá trình huấn luyện. Điều này giúp giảm overfitting và làm cho mô hình trở nên tổng quát hơn.

tf.keras.layers.BatchNormalization: Lớp này thực hiện batch normalization, giúp ổn định và tăng tốc quá trình huấn luyện của mạng nơ-ron. Chúng chuẩn hóa đầu ra của các layer trước khi được đưa vào các activation function, giảm hiện tượng vanishing gradient và tăng tốc độ hội tụ của mô hình.

tf.keras.losses: Module này chứa nhiều hàm loss function thường được sử dụng, như Mean Squared Error (MSE), Categorical Crossentropy, Binary Crossentropy, ...

tf.keras.optimizers: Module này chứa các optimizer thường được sử dụng cho việc tối ưu hóa mạng nơ-ron, bao gồm SGD, Adam, RMSprop, ...


In [55]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Build model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(train_images, train_labels, epochs=5)

# Evaluate model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2972 - accuracy: 0.9126
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1414 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1082 - accuracy: 0.9678
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0874 - accuracy: 0.9725
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0762 - accuracy: 0.9775
Test accuracy: 0.9775000214576721
